In [ ]:
# %% [markdown]
# # Reward Engineering Pilot: Analysis Notebook
# 
# This notebook loads the final evaluation logs from the experiment, calculates key performance metrics, and generates visualizations for the paper.

# %%
# Step 1: Import Libraries and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import json
from pandas import json_normalize
from scipy import stats

# Setup plotting style
sns.set_theme(style="whitegrid")

# Define the path to the log directory from your settings
# (You might need to adjust the path depending on where you run the notebook)
LOG_DIR = "../logs" 

# %% [markdown]
# ## Step 2: Data Loading and Preprocessing
# Find all `_eval.jsonl` files, load them, and merge them into a single Pandas DataFrame.

# %%
def load_all_eval_logs(log_dir: str) -> pd.DataFrame:
    """Finds and loads all evaluation logs into a single DataFrame."""
    log_files = glob.glob(os.path.join(log_dir, "*_eval.jsonl"))
    if not log_files:
        print("No evaluation log files found.")
        return pd.DataFrame()

    all_logs = []
    for file in log_files:
        with open(file, 'r', encoding='utf-8') as f:
            for line in f:
                all_logs.append(json.loads(line))
    
    # Flatten the nested JSON structure into columns
    df = json_normalize(all_logs)
    return df

df = load_all_eval_logs(LOG_DIR)

# --- Data Cleaning and Feature Engineering ---
if not df.empty:
    # Convert relevant columns to numeric types
    for col in ['reward.correctness_score', 'reward.complexity_score']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Create a single 'final_reward' column for easier analysis
    def calculate_final_reward(row):
        # This logic must match the one in your trial_runner.py
        if row['config.condition'] in ['B', 'D']:
            if row['reward.goal_alignment'] and row['reward.whw_description_rule']:
                return (row['reward.correctness_score'] * 0.35) + \
                       (row['reward.complexity_score'] * 0.65)
            else:
                return 0.0
        else: # A, C
            return row['reward.correctness_score']
    
    df['final_reward'] = df.apply(calculate_final_reward, axis=1)

    # Define "Reward Hacking"
    # A simple definition: The model failed on correctness but still received partial reward signals.
    df['is_reward_hack'] = (df['eval.correctness_score'] == 0) & \
                            ( (df['eval.complexity_score'] > 0) | (df['reward.goal_alignment'] == True) )

    print(f"Loaded and processed {len(df)} total records.")
    display(df.head())

# %% [markdown]
# ## Step 3: Core Metric Calculation
# Group data by model and condition to calculate the main metrics.

# %%
if not df.empty:
    # Group by model and condition
    grouped = df.groupby(['config.model_name', 'config.condition'])

    # --- Calculate Metrics ---
    metrics = grouped.agg(
        avg_final_reward=('final_reward', 'mean'),
        success_rate=('eval.correctness_score', 'mean'),
        avg_complexity_score=('eval.complexity_score', 'mean'),
        whw_fidelity=('eval.whw_condition', lambda x: x.mean(skipna=True)), # Skipna for A/C
        reward_hack_rate=('is_reward_hack', 'mean'),
        n_samples=('final_reward', 'count')
    ).round(3)

    print("--- Key Performance Metrics ---")
    display(metrics)

# %% [markdown]
# ## Step 4: Visualization
# Generate plots for the paper.

# %%
if not df.empty:
    # --- Plot 1: Average Final Reward by Condition and Model ---
    plt.figure(figsize=(12, 7))
    sns.barplot(
        data=df,
        x='config.model_name',
        y='final_reward',
        hue='config.condition',
        palette='viridis'
    )
    plt.title('Average Final Reward by Model and Condition', fontsize=16)
    plt.ylabel('Average Reward Score')
    plt.xlabel('Model Name')
    plt.xticks(rotation=15)
    plt.legend(title='Condition')
    plt.tight_layout()
    plt.savefig('results/average_reward_by_condition.png')
    plt.show()

    # --- Plot 2: Success Rate (Correctness) by Condition and Model ---
    plt.figure(figsize=(12, 7))
    sns.barplot(
        data=df,
        x='config.model_name',
        y='eval.correctness_score',
        hue='config.condition',
        palette='plasma'
    )
    plt.title('Success Rate (Correctness) by Model and Condition', fontsize=16)
    plt.ylabel('Success Rate')
    plt.xlabel('Model Name')
    plt.xticks(rotation=15)
    plt.legend(title='Condition')
    plt.tight_layout()
    plt.savefig('results/success_rate_by_condition.png')
    plt.show()
    
    # --- Plot 3: Reward Hacking Rate (for B and D conditions) ---
    df_bd = df[df['config.condition'].isin(['B', 'D'])]
    if not df_bd.empty:
        plt.figure(figsize=(10, 6))
        sns.barplot(
            data=df_bd,
            x='config.model_name',
            y='is_reward_hack',
            hue='config.condition',
            palette='coolwarm'
        )
        plt.title('Reward Hacking Rate by Model (Conditions B & D)', fontsize=16)
        plt.ylabel('Hack Rate')
        plt.xlabel('Model Name')
        plt.xticks(rotation=15)
        plt.legend(title='Condition')
        plt.tight_layout()
        plt.savefig('results/reward_hacking_rate.png')
        plt.show()

# %% [markdown]
# ## Step 5: Statistical Analysis (Example)
# Calculate bootstrap confidence intervals for a key metric.

# %%
if not df.empty:
    # Example: Bootstrap 95% CI for the average reward of o4-mini in Condition B
    o4_mini_b_rewards = df[
        (df['config.model_name'] == 'o4-mini-2025-04-16') & 
        (df['config.condition'] == 'B')
    ]['final_reward'].dropna()

    if not o4_mini_b_rewards.empty:
        # Convert to numpy array for bootstrapping
        data = o4_mini_b_rewards.to_numpy()
        
        # Create bootstrap distribution
        bootstrap_means = [np.mean(np.random.choice(data, size=len(data), replace=True)) for _ in range(1000)]
        
        # Calculate confidence interval
        confidence_interval = np.percentile(bootstrap_means, [2.5, 97.5])
        
        print("--- Example: Bootstrap 95% CI ---")
        print(f"Model: o4-mini, Condition: B")
        print(f"Mean Reward: {np.mean(data):.3f}")
        print(f"95% Confidence Interval: [{confidence_interval[0]:.3f}, {confidence_interval[1]:.3f}]")